## Experiments on Instrument Detection Models

11/30/2020

Evaluation Metrics: F1 score, Expected Calibration Error (ECE)

In [8]:
# set up autoreload 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### define data paths

In [9]:
# define paths to datamodules 
dm_paths = {
    'audio': 
    '/home/hugo/CHONK/data/mdb-hop-0.25-chunk-1-AUGMENTED/splits-85-15-REMAPPED-testclean', 
    
    'embeddings-512': 
    '/home/hugo/CHONK/data/mdb-hop-0.25-chunk-1-AUGMENTED-EMBEDDINGS-512/splits-85-15-REMAPPED-testclean', 

    'embeddings-6144':
    '/home/hugo/CHONK/data/mdb-hop-0.25-chunk-1-AUGMENTED-EMBEDDINGS-6144/splits-85-15-REMAPPED-testclean'}


### define a run_trial function

In [10]:
%%capture
from instrument_recognition.task import InstrumentDetectionTask, train_instrument_detection_model
from instrument_recognition.models.zoo import load_model
from instrument_recognition.datasets import load_datamodule
import os
import yaml

def run_trial(exp_dict, test_only=False):

    # load the datamodule
    print(f'loading datamodule...')
    dm = load_datamodule(path_to_data=exp_dict['path_to_data'], 
                         batch_size=exp_dict['batch_size'], 
                         num_workers=exp_dict['num_workers'],
                         use_embeddings=exp_dict['use_embeddings'])
    
    # get classlist and number of classes
    classlist = dm.get_classes()
    num_output_units = len(classlist)
    print(f'classlist is: {classlist}')

    # load model
    print(f'loading model...')
    model = load_model(model_name=exp_dict['model_name'], 
                       output_units=num_output_units, 
                       dropout=exp_dict['dropout'], 
                       random_seed=exp_dict['random_seed'])
    
    # build task
    print(f'building task...')
    task = InstrumentDetectionTask(model, dm, 
                            max_epochs=exp_dict['max_epochs'],
                            learning_rate=exp_dict['learning_rate'], 
                            weighted_cross_entropy=exp_dict['weighted_cross_entropy'], 
                            mixup=exp_dict['mixup'],
                            mixup_alpha=exp_dict['mixup_alpha'], 
                            log_epoch_metrics=True)
    
    # run train fn and get back test results
    print(f'running task')
    result = train_instrument_detection_model(task, 
                                    name=exp_dict['name'], 
                                    version=exp_dict['version'], 
                                    gpuid=exp_dict['gpuid'], 
                                    max_epochs=exp_dict['max_epochs'],
                                    random_seed=exp_dict['random_seed'], 
                                    log_dir=exp_dict['log_dir'],
                                    test_only=test_only,
                                    **exp_dict['trainer_kwargs'])

    # save exp_dict to yaml file for easy reloading
    with open(os.path.join(exp_dict['log_dir'], 'exp_dict.yaml'), 'w') as f:
        yaml.dump(exp_dict, f)
    
    result = result[0]

    return result

### define fixed hyperparameters

we won't change these (except for the batch size in raw audio models)

In [11]:
import datetime
import os
now = datetime.datetime.now().strftime("%d.%m.%Y")
log_dir = os.path.join('logs', f'experiment-uncertainty-{now}')
# log_dir = os.path.join('logs', 'experiment-02.12.2020')

# define experiment
FIXED = dict(batch_size=128*3, num_workers=20, learning_rate=3e-4,
             weighted_cross_entropy=True, dropout=0.5, random_seed=20, 
             max_epochs=80, log_dir=log_dir,  version=0,  gpuid=-1,
             trainer_kwargs={}) 
    
def make_trial(trial_dict):
    """ creates a dict with fixed hyperparams
    and then adds trial_dict on top
    """
    exp_dict = dict(FIXED) 
    exp_dict.update(trial_dict)
    return exp_dict


### define all trials

In [12]:
trials = [
    dict(name='openl3-mlp-ERM',
    path_to_data=dm_paths['embeddings-6144'],
    use_embeddings= True, 
    model_name='mlp-6144',
    mixup=False,
    mixup_alpha=0), 
    
    dict(name='openl3-mlp-MIXUP-alpha=0.2',
    path_to_data=dm_paths['embeddings-6144'],
    use_embeddings= True, 
    model_name='mlp-6144',
    mixup=True,
    mixup_alpha=0.2),
    
    dict(name='openl3-mlp-MIXUP-alpha=0.4',
    path_to_data=dm_paths['embeddings-6144'],
    use_embeddings= True, 
    model_name='mlp-6144',
    mixup=True,
    mixup_alpha=0.4), 
    
    dict(name='openl3-mlp-ERM-ensemble-4',
    path_to_data=dm_paths['embeddings-6144'],
    use_embeddings= True, 
    model_name='mlp-6144-ensemble-4',
    mixup=False,
    mixup_alpha=0), 
    
    dict(name='openl3-mlp-MIXUP-alpha=0.2-ensemble-4',
    path_to_data=dm_paths['embeddings-6144'],
    use_embeddings= True, 
    model_name='mlp-6144-ensemble-4',
    mixup=True,
    mixup_alpha=0.2),
    

]
trials = {t['name']: make_trial(t) for t in trials}

## train models!! :) 

In [13]:
# run me too see what trials I can run
for k in trials:
    print(k)

openl3-mlp-ERM
openl3-mlp-MIXUP-alpha=0.2
openl3-mlp-MIXUP-alpha=0.4
openl3-mlp-ERM-ensemble-4
openl3-mlp-MIXUP-alpha=0.2-ensemble-4


In [ ]:
# %%capture cap
run_trial(trials['openl3-mlp-ERM'])

loading datamodule...
medley db path: /home/hugo/data/medleydb


/home/hugo/lab/venv/lib/python3.6/site-packages/medleydb/__init__.py:65: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.

/home/hugo/lab/venv/lib/python3.6/site-packages/medleydb/__init__.py:73: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.

INFO:lightning:GPU available: True, used: True
INFO:lightning:TPU available: False, using: 0 TPU cores
INFO:lightning:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
INFO:lightning:Using native 16bit precision.


train entries: 288692
val entries: 14005
classlist is: ['acoustic guitar', 'auxiliary percussion', 'brass section', 'cello', 'clean electric guitar', 'distorted electric guitar', 'double bass', 'drum set', 'electric bass', 'female singer', 'male singer', 'oboe', 'piano', 'synthesizer', 'tack piano', 'trumpet', 'vibraphone', 'viola', 'violin', 'vocalists']
loading model...
building task...
running task


/home/hugo/lab/venv/lib/python3.6/site-packages/torch/nn/parallel/data_parallel.py:30: UserWarning:


    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.

INFO:lightning:
   | Name       | Type        | Params
--------------------------------------------
0  | model      | MLP6144     | 3 M   
1  | model.fc   | Sequential  | 3 M   
2  | model.fc.0 | BatchNorm1d | 12 K  
3  | model.fc.1 | Linear      | 3 M   
4  | model.fc.2 | ReLU        | 0     
5  | model.fc.3 | Dropout     | 0     
6  | model.fc.4 | BatchNorm1d | 1 K   
7  | model.fc.5 | Linear      | 65 K  
8  | model.fc.6 | ReLU        | 0     
9  | model.fc.7 | Dropout     | 0     
10 | model.fc.8 | BatchNorm1d | 256   
11 | model.fc.9 | Linear      | 2 K   


Test tube created git tag: tt_openl3-mlp-ERM_v0


...

INFO:lightning:Epoch 2: loss/val reached 1.01792 (best 0.90700), saving model to /home/hugo/lab/mono_music_sed/instrument_recognition/notebooks/logs/experiment-uncertainty-04.12.2020/openl3-mlp-ERM/version_0/checkpoints/epoch=02-loss_val=1.02.ckpt as top 3


INFO:lightning:Epoch 3: loss/val was not in top 3


INFO:lightning:Epoch 4: loss/val reached 0.94855 (best 0.90700), saving model to /home/hugo/lab/mono_music_sed/instrument_recognition/notebooks/logs/experiment-uncertainty-04.12.2020/openl3-mlp-ERM/version_0/checkpoints/epoch=04-loss_val=1.00.ckpt as top 3


INFO:lightning:Epoch 5: loss/val reached 1.00980 (best 0.90700), saving model to /home/hugo/lab/mono_music_sed/instrument_recognition/notebooks/logs/experiment-uncertainty-04.12.2020/openl3-mlp-ERM/version_0/checkpoints/epoch=05-loss_val=1.01.ckpt as top 3


INFO:lightning:Epoch 6: loss/val was not in top 3


In [ ]:
# %%capture cap
run_trial(trials['openl3-mlp-MIXUP-alpha=0.2'])

In [ ]:
# %%capture cap
run_trial(trials['openl3-mlp-MIXUP-alpha=0.4'])

In [ ]:
# %%capture cap
run_trial(trials['openl3-mlp-ERM-ensemble-4'])


loading datamodule...
medley db path: /home/hugo/data/medleydb


/home/hugo/lab/venv/lib/python3.6/site-packages/medleydb/__init__.py:65: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.

/home/hugo/lab/venv/lib/python3.6/site-packages/medleydb/__init__.py:73: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.

/home/hugo/lab/venv/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning:

Checkpoint directory /home/hugo/lab/mono_music_sed/instrument_recognition/notebooks/logs/experiment-uncertainty-04.12.2020/openl3-mlp-ERM-ensemble-4/version_0/checkpoints exists and is not empty. With save_top_k=3, all files in this directory will be deleted when a checkpoint is saved!

INFO:lightning:GPU available: True, used: True
INFO:lightning:TPU available: False, using: 0 TPU cores
INFO:lightning:LOCAL_RANK:

train entries: 288692
val entries: 14005
classlist is: ['acoustic guitar', 'auxiliary percussion', 'brass section', 'cello', 'clean electric guitar', 'distorted electric guitar', 'double bass', 'drum set', 'electric bass', 'female singer', 'male singer', 'oboe', 'piano', 'synthesizer', 'tack piano', 'trumpet', 'vibraphone', 'viola', 'violin', 'vocalists']
loading model...
building task...
running task
found checkpoint: logs/experiment-uncertainty-04.12.2020/openl3-mlp-ERM-ensemble-4/version_0/checkpoints/epoch=08-loss_val=0.99.ckpt


/home/hugo/lab/venv/lib/python3.6/site-packages/torch/nn/parallel/data_parallel.py:30: UserWarning:


    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.

INFO:lightning:
   | Name                 | Type        | Params
------------------------------------------------------
0  | model                | Ensemble    | 12 M  
1  | model.members        | ModuleList  | 12 M  
2  | model.members.0      | MLP6144     | 3 M   
3  | model.members.0.fc   | Sequential  | 3 M   
4  | model.members.0.fc.0 | BatchNorm1d | 12 K  
5  | model.members.0.fc.1 | Linear      | 3 M   
6  | model.members.0.fc.2 | ReLU        | 0     
7  | model.members.0.fc.3 | Dropout     | 0     
8  | model.members.0.fc.4 | BatchNorm1d | 1 K   
9  | model.members.0.fc.5 | Linear      | 65 K  
10 | model.membe

Test tube created git tag: tt_openl3-mlp-ERM-ensemble-4_v0


/home/hugo/lab/venv/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:64: UserWarning:

Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.



INFO:lightning:Epoch 9: loss/val reached 1.22595 (best 1.22595), saving model to /home/hugo/lab/mono_music_sed/instrument_recognition/notebooks/logs/experiment-uncertainty-04.12.2020/openl3-mlp-ERM-ensemble-4/version_0/checkpoints/epoch=09-loss_val=1.22.ckpt as top 3


INFO:lightning:Epoch 10: loss/val reached 1.28017 (best 1.22595), saving model to /home/hugo/lab/mono_music_sed/instrument_recognition/notebooks/logs/experiment-uncertainty-04.12.2020/openl3-mlp-ERM-ensemble-4/version_0/checkpoints/epoch=10-loss_val=1.28.ckpt as top 3


In [ ]:
# %%capture cap
run_trial(trials['openl3-mlp-MIXUP-alpha=0.2-ensemble-4'])

loading datamodule...


INFO:lightning:GPU available: True, used: True
INFO:lightning:TPU available: False, using: 0 TPU cores
INFO:lightning:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
INFO:lightning:Using native 16bit precision.


train entries: 288692
val entries: 14005
classlist is: ['acoustic guitar', 'auxiliary percussion', 'brass section', 'cello', 'clean electric guitar', 'distorted electric guitar', 'double bass', 'drum set', 'electric bass', 'female singer', 'male singer', 'oboe', 'piano', 'synthesizer', 'tack piano', 'trumpet', 'vibraphone', 'viola', 'violin', 'vocalists']
loading model...
building task...
running task


INFO:lightning:
   | Name                 | Type        | Params
------------------------------------------------------
0  | model                | Ensemble    | 12 M  
1  | model.members        | ModuleList  | 12 M  
2  | model.members.0      | MLP6144     | 3 M   
3  | model.members.0.fc   | Sequential  | 3 M   
4  | model.members.0.fc.0 | BatchNorm1d | 12 K  
5  | model.members.0.fc.1 | Linear      | 3 M   
6  | model.members.0.fc.2 | ReLU        | 0     
7  | model.members.0.fc.3 | Dropout     | 0     
8  | model.members.0.fc.4 | BatchNorm1d | 1 K   
9  | model.members.0.fc.5 | Linear      | 65 K  
10 | model.members.0.fc.6 | ReLU        | 0     
11 | model.members.0.fc.7 | Dropout     | 0     
12 | model.members.0.fc.8 | BatchNorm1d | 256   
13 | model.members.0.fc.9 | Linear      | 2 K   
14 | model.members.1      | MLP6144     | 3 M   
15 | model.members.1.fc   | Sequential  | 3 M   
16 | model.members.1.fc.0 | BatchNorm1d | 12 K  
17 | model.members.1.fc.1 | Linear      | 3 M  

Test tube created git tag: tt_openl3-mlp-MIXUP-alpha=0.2-ensemble-4_v0


...

INFO:lightning:Epoch 14: loss/val was not in top 3


INFO:lightning:Epoch 15: loss/val was not in top 3


INFO:lightning:Epoch 16: loss/val was not in top 3


INFO:lightning:Epoch 17: loss/val was not in top 3


INFO:lightning:Epoch 18: loss/val was not in top 3


## test models :)

In [ ]:
import traceback 
import pandas as pd

scoreboard = []
for exp in exp_dicts:
    print(100*'-', '\n', 100*'-')
    try:
        if not exp['done']:
            results = run_trial(exp)
            exp['done'] = True
        else:
            print(f'skipping {exp["name"]}')
    except:
        print(f'exp: {exp["name"]} failed.')
        traceback.print_exc(limit=None, file=None, chain=True)
        results = {'error': True}
        
    score = dict(exp)
    score.update(results)
    scoreboard.append(score)
    
scoreboard_df = pd.DataFrame(scoreboard)

In [ ]:
%store

In [ ]:
scoreboard_df.to_csv(os.path.join(log_dir, 'scoreboard.csv'))
with open(os.path.join(log_dir, 'output.txt'),  'w') as f:
    f.write(cap.stdout)